In [8]:
import cv2
import glob
import os
import numpy as np
from os.path import isfile, join

# Main Directory
input_path = r'E:/Research_medical/dataset_medial/'
output_path = r'I:/f_p/'

traintest_folder = os.listdir(input_path)


def laplacian(image):
    # Apply Laplacian sharpening
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    sharpened_image = np.uint8(np.clip(gray - 0.5 * laplacian, 0, 255))

    # Apply unsharp masking
    blurred = cv2.GaussianBlur(gray, (0, 0), 3)
    unsharp_mask = cv2.addWeighted(gray, 2, blurred, -1, 0)
    sharpened_image = np.uint8(np.clip(unsharp_mask, 0, 255))

    return sharpened_image


def highPassFiltering(image):
    # Apply high-pass filtering
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32)
    filtered_image = cv2.filter2D(gray, -1, kernel)

    return filtered_image


def kalman_filter(image):
    # Apply Kalman Filter to smooth the image
    kalman = cv2.KalmanFilter(4, 2)
    kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
    kalman.processNoiseCov = 1e-5 * np.eye(4, dtype=np.float32)
    kalman.measurementNoiseCov = 1e-3 * np.eye(2, dtype=np.float32)

    # Convert image to float32 for Kalman Filter
    image = image.astype(np.float32)

    # Initialize Kalman Filter state
    state = np.array([[image[0, 0]], [image[0, 1]], [0], [0]], dtype=np.float32)

    for i in range(1, image.shape[0]):
        # Predict
        predicted_state = kalman.predict()

        # Correct using measurement
        measurement = np.array([[image[i, 0]], [image[i, 1]]], dtype=np.float32)
        corrected_state = kalman.correct(measurement)

        # Update image with corrected state
        image[i, 0] = corrected_state[0, 0]
        image[i, 1] = corrected_state[1, 0]

    return image


for segments in traintest_folder:
    
    print("Working on {} ----------------------------------------------".format(segments))
    # Train Test Val Folders
    segment_path = join(input_path, segments)
    output_segment_path = join(output_path,segments)
    # Create Folder
    if not os.path.exists(output_segment_path):
        os.mkdir(output_segment_path)
        
    # Indiidual Folders
    classes = os.listdir(segment_path)
    for class_folder in classes:
        
        print("Working on {} ------------------".format(class_folder))
        input_class_folder = join(segment_path, class_folder)
        output_class_folder = join(output_segment_path, class_folder)
        # Create Folder
        if not os.path.exists(output_class_folder):
            os.mkdir(output_class_folder)

        images = os.listdir(input_class_folder)
        
        for i in images:
            
            # Read Image
            image = cv2.imread(join(input_class_folder, i), cv2.IMREAD_COLOR)
            # Convert the image to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
            # Laplacian Transformation
            transformed_image = laplacian(image)
            # High Pass Filtering
            transformed_image1 = highPassFiltering(transformed_image)
             # kalman Filtering
            transformed_image2 =  kalman_filter(transformed_image1)
            
            
            
            filename = join(output_class_folder, i)
            cv2.imwrite("{}".format(filename), transformed_image)

Working on auto_test ----------------------------------------------
Working on 0_healthy ------------------
Working on 1_doubtful ------------------
Working on 2_minimal ------------------
Working on 3_moderate ------------------
Working on 4_severe ------------------
Working on test ----------------------------------------------
Working on 0_healthy ------------------
Working on 1_doubtful ------------------
Working on 2_minimal ------------------
Working on 3_moderate ------------------
Working on 4_severe ------------------
Working on train ----------------------------------------------
Working on 0_healthy ------------------
Working on 1_doubtful ------------------
Working on 2_minimal ------------------
Working on 3_moderate ------------------
Working on 4_severe ------------------
Working on val ----------------------------------------------
Working on 0_healthy ------------------
Working on 1_doubtful ------------------
Working on 2_minimal ------------------
Working on 3_modera